In [8]:
import torch
import torch.nn as nn
#import torchvision.datasets as dsets
#import torchvision.transforms as transforms
import random
import matplotlib.pyplot as plt
import csv
import sys
import numpy as np
import time
import copy
import math as m
import torch.nn.init

gpu_num = 0

In [9]:
device = ('cuda'+':'+ str(gpu_num)) if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == ('cuda'+ ':' + str(gpu_num)) :
    torch.cuda.manual_seed_all(777)

In [10]:
learning_rate   = 0.001
training_epochs = 500
batchsize       = 1000000 #full
rad2deg         = 57.29577951
deg2rad         = 0.01745329252

In [11]:
count = 0
trainFilename = '/media/workstation2/HDD/Daseon/sim/Missile_Simulator_chogandan_2dim_PNG/ITCG_trainset_Vm20000shoot_360.csv'
f = open(trainFilename,'r')
Reader = csv.reader(f)
trainDataset_raw = []
for row in Reader:
    trainDataset_raw.append(row)
trainDataset_raw = np.array(trainDataset_raw, dtype='float32')
f.close()

In [12]:
trainDataset_raw
trainMean = np.mean(trainDataset_raw, axis=0)
trainStd  = np.std(trainDataset_raw, axis=0)
trainDataset = (trainDataset_raw - trainMean)/trainStd
print('mean ',np.mean(trainDataset_raw, axis=0))
print('std  ',np.std(trainDataset_raw, axis=0))

mean  [ 1.0429623e+04 -3.8220634e+00  1.2230603e+02  1.9110317e+00
  6.3654346e+01]
std   [1.2492353e+04 3.3081369e+00 1.5931599e+02 1.3973013e+00 8.8367859e+01]


In [13]:
trainDataset.shape[0]

70233126

In [14]:
def get_batch(batchsize, dataset):
    if(trainDataset.shape[0]==batchsize):
        batchNumpy = dataset
    else:
        batchNumpy = dataset[np.random.choice(dataset.shape[0], batchsize, replace=False), 0:]
    
    return torch.from_numpy(batchNumpy[0:,0:-1]), torch.from_numpy(batchNumpy[0:,-1:])


In [15]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        
        self.fc1     = nn.Linear(4, 150, bias = True)    
        self.relu1   = nn.ReLU()
        self.fc2     = nn.Linear(150, 300, bias = True) 
        self.relu2   = nn.ReLU()
        self.fc3     = nn.Linear(300, 500, bias = True)
        self.relu3   = nn.ReLU()
        self.fc4     = nn.Linear(500, 300, bias = True)
        self.relu4   = nn.ReLU()
        self.fc5     = nn.Linear(300, 150, bias = True)
        self.relu5   = nn.ReLU()
        self.fc6     = nn.Linear(150, 1, bias = True)
        
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        torch.nn.init.xavier_uniform_(self.fc3.weight)
        torch.nn.init.xavier_uniform_(self.fc4.weight)
        torch.nn.init.xavier_uniform_(self.fc5.weight)
        torch.nn.init.xavier_uniform_(self.fc6.weight)
        
    def forward(self, x):
        
        out = self.fc1(x)
        out = self.relu1(out)
        
        out = self.fc2(out)
        out = self.relu2(out)
        
        out = self.fc3(out)
        out = self.relu3(out)
        
        out = self.fc4(out)
        out = self.relu4(out)
        
        out = self.fc5(out)
        out = self.relu5(out)
        
        out = self.fc6(out)
        
        return out

In [16]:
model = DNN().to(device)
criterion = nn.SmoothL1Loss().to(device)


In [17]:
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
training_epochs = 100000
for epoch in range(training_epochs):
    avg_cost = 0
    
    for i in range(int(trainDataset.shape[0]/batchsize)):
        tbbb = time.time()
        batch_input, batch_label = get_batch(batchsize, trainDataset)
        print(time.time()-tbbb)
        #print('batchpass', i)
        batch_input           = batch_input.to(device) 
        batch_label           = batch_label.to(device) 
        
        optimizer.zero_grad()
        hypothesis = model(batch_input)
        
        cost = criterion(hypothesis, batch_label)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost /(int(trainDataset.shape[0]/batchsize))
        
    count = 0       
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))

4.900389194488525
4.889747619628906
4.864917755126953
4.852269887924194
4.85170578956604
4.892646789550781
4.8533616065979
4.8824872970581055
4.882595777511597
4.891183137893677
4.879257440567017
4.874614715576172
4.873504877090454
4.87442421913147
4.8743157386779785
4.853069305419922
4.84692120552063
4.849628448486328
4.848248243331909
4.865524053573608
4.891363143920898
4.882126808166504
4.876296043395996
4.957646131515503
4.903357028961182
4.906477689743042
4.86174464225769
4.893558502197266
4.880126714706421
4.855976343154907
4.887985467910767
4.878361463546753
4.87810206413269
4.882338285446167
4.873610496520996
4.861278772354126
4.847163200378418
4.847432613372803
4.879407644271851
4.873044967651367
4.871598482131958
4.862567901611328
4.910052299499512
4.914326429367065
4.898504972457886
4.948851108551025
4.946192502975464
4.87897801399231
4.8949854373931885
4.897550821304321
4.886205673217773
4.8536906242370605
4.851547002792358
4.912379264831543
4.854130268096924
4.885519027709

In [ ]:
testFilename = '/home/workstation2/Daseon/DeepMonopulse/datTest_noNoise.csv'
ff = open(testFilename,'r')
Reader = csv.reader(ff)
testDataset_raw = []
for row in Reader:
    testDataset_raw.append(row)
testDataset_raw = np.array(testDataset_raw, dtype='float32')
ff.close()

testDataset = (testDataset_raw - trainMean)/trainStd
    

In [ ]:
with torch.no_grad():
    index = np.random.choice(testDataset.shape[0], 5, replace=False)
    inference = model(torch.from_numpy(testDataset[index, :-2]).to(device))
    answer = testDataset_raw[index, -2:]
    
    inference = inference.to('cpu')
    print('predic : ',(inference*trainStd[-2:]+trainMean[-2:])*rad2deg)
    print('Realvl : ',answer*rad2deg)

In [ ]:
kkk = [[1,1,3],[3,8,5],[6,7,7],[8,6,2],[8,1,1]]
kkk = np.array(kkk)

In [ ]:
kkk.random.choice(5,3)

In [ ]:
kkk[np.array([1,3]),0:-2]

In [ ]:
mm = np.mean(kkk, axis=0)
mm

In [ ]:
ss = np.std(kkk, axis=0)
ss

In [ ]:
inference

In [1]:
torch.save(model.state_dict(), './360WEIGHT_'+str(avg_cost.item()))

NameError: name 'torch' is not defined

In [ ]:
print(avg_cost.item())